In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
import sys
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats

In [2]:
# Saving id variables to create final submission
test = pd.read_csv('test.csv')
ids_test = test['User_ID'].copy()
product_ids_test = test['Product_ID'].copy()

In [3]:
train_feature = pd.read_csv('train_feature.csv')
test_feature  = pd.read_csv('test_feature.csv')

In [4]:
train_feature.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Purchase,...,Product_ID_MinPrice,Product_ID_MaxPrice,Product_ID_MeanPrice,Product_ID_25PercPrice,Product_ID_75PercPrice,Product_Category_1_MinPrice,Product_Category_1_MaxPrice,Product_Category_1_MeanPrice,Product_Category_1_25PercPrice,Product_Category_1_75PercPrice
0,0,672,0,0,10,0,2,0,3,8370.0,...,2648.0,13716.0,11870.863436,10804.00,13475.50,2638.0,13717.0,10096.705734,8198.0,13211.0
1,0,2376,0,0,10,0,2,0,1,15200.0,...,3880.0,19701.0,16304.030981,15312.00,19264.00,3790.0,19708.0,13606.218596,11546.0,15812.0
2,0,852,0,0,10,0,2,0,12,1422.0,...,343.0,1776.0,1237.892157,1043.75,1423.75,342.0,1778.0,1350.859894,1071.0,1723.0
3,0,828,0,0,10,0,2,0,12,1057.0,...,365.0,1778.0,1455.140762,1378.00,1736.00,342.0,1778.0,1350.859894,1071.0,1723.0
4,1,2734,1,6,16,2,4,0,8,7969.0,...,3920.0,10073.0,7692.763547,6174.50,8082.50,1939.0,10082.0,7498.958078,6036.0,9722.0


In [5]:
test_feature.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Age_count,...,Product_ID_MinPrice,Product_ID_MaxPrice,Product_ID_MeanPrice,Product_ID_25PercPrice,Product_ID_75PercPrice,Product_Category_1_MinPrice,Product_Category_1_MaxPrice,Product_Category_1_MeanPrice,Product_Category_1_25PercPrice,Product_Category_1_75PercPrice
0,3,1196,1,4,7,1,2,1,1,45701,...,3933.0,19708.0,15781.118590,12067.50,19248.50,3790.0,19708.0,13606.218596,11546.0,15812.0
1,8,1043,1,2,17,2,0,0,3,219587,...,3077.0,13714.0,11746.665354,10731.25,13493.25,2638.0,13717.0,10096.705734,8198.0,13211.0
2,9,2764,0,3,1,1,4,1,5,110013,...,1734.0,8900.0,5731.338028,5147.00,7027.00,1713.0,8907.0,6240.088178,5242.0,7156.0
3,9,1358,0,3,1,1,4,1,4,110013,...,813.0,3526.0,1943.266667,1459.00,2129.50,684.0,3556.0,2329.659491,2058.0,2837.0
4,10,529,0,2,1,2,1,0,4,219587,...,684.0,3556.0,2585.590829,2115.00,3421.50,684.0,3556.0,2329.659491,2058.0,2837.0


<h2>Modelling</h2>

In [6]:
#Labelling dataset
X = train_feature[:]
y = train_feature['Purchase']

# Dropping Purchase from Feature Matrix i.e train data which is now stored in X
X.drop(["Purchase"], axis=1, inplace=True)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
!pip install xgboost

    100% |################################| 1.2MB 1.2MB/s eta 0:00:01
  Running setup.py bdist_wheel for xgboost ... done
  Stored in directory: /home/nbuser/.cache/pip/wheels/5e/c1/d6/522af54e5cc001fad4dd855117f8bf61b11d56443e06672e26
Successfully built xgboost


In [17]:
import xgboost as xgb
dtrain = xgb.DMatrix(X.values, label = y, missing=np.nan)

param = {'objective': 'reg:linear', 'booster': 'gbtree', 'silent': 1,
         'max_depth': 10, 'eta': 0.1, 'nthread': 4,
         'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 20,
         'max_delta_step': 0, 'gamma': 0}

num_round = 690

In [18]:
seeds = [1122, 2244, 3366, 4488, 5500]
test_pred = np.zeros((len(test_feature), len(seeds)))

In [19]:
for run in range(len(seeds)):
    sys.stdout.write("\rXGB RUN:{}/{}".format(run+1, len(seeds)))
    sys.stdout.flush()
    param['seed'] = seeds[run]
    clf = xgb.train(param, dtrain, num_round)
    dtest = xgb.DMatrix(test_feature.values, missing=np.nan)
    test_pred[:, run] = clf.predict(dtest)

XGB RUN:5/5

In [20]:
test_pred = np.mean(test_pred, axis=1)

In [21]:
# Submission file

submit = pd.DataFrame({'User_ID': ids_test, 'Product_ID': product_ids_test, 'Purchase': test_pred})
submit = submit[['User_ID', 'Product_ID', 'Purchase']]

In [22]:
submit.ix[submit['Purchase'] < 0, 'Purchase'] = 12  # changing min prediction to min value in train
submit.to_csv("final_solutionBinga.csv", index=False)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':
Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x7f4e0f90d748>>
Traceback (most recent call last):
  File "/home/nbuser/anaconda3_501/lib/python3.6/site-packages/xgboost/core.py", line 324, in __del__
    _check_call(_LIB.XGDMatrixFree(self.handle))
AttributeError: 'DMatrix' object has no attribute 'handle'
